# Pandas Intro — Part 2
Practical data-wrangling patterns:
- Load a CSV with **missing values** and handle them
- **Feature engineering** (new columns)
- **Scaling** numeric features
- **Joins/Merges** between tables
- A couple of Matplotlib charts

Dataset: Iris (classic) with a few injected missing values for demo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Paths (adjust to your repo layout as needed)
CSV_PATH = "ml_projects/data/iris_with_missing.csv"  # put this csv under your repo


In [ ]:
# Load the CSV (created for you alongside this notebook)
df = pd.read_csv(CSV_PATH)
df.head()

In [ ]:
# Missing values overview
print("Shape:", df.shape)
print("\nMissing per column:")
print(df.isna().sum())

In [ ]:
# Simple missing-value handling strategies:
# 1) Drop rows with any missing values (for demo only — can lose data)
dropped = df.dropna()
print("After dropna:", dropped.shape)

# 2) Impute with column means for numeric features
num_cols = [c for c in df.columns if df[c].dtype != 'object']
imputed = df.copy()
for c in num_cols:
    imputed[c] = imputed[c].fillna(imputed[c].mean())

print("Imputed missing remaining:", imputed.isna().sum().sum())  # should be 0
imputed.head()

In [ ]:
# Feature engineering: simple areas
fe = imputed.copy()
fe['sepal_area'] = fe['sepal length (cm)'] * fe['sepal width (cm)']
fe['petal_area'] = fe['petal length (cm)'] * fe['petal width (cm)']
fe[['sepal_area','petal_area']].describe()

In [ ]:
# Scaling numeric features (standardization)
from sklearn.preprocessing import StandardScaler

num_cols = [c for c in fe.columns if fe[c].dtype != 'object' and c != 'species_index']
scaler = StandardScaler()
scaled_vals = scaler.fit_transform(fe[num_cols])
scaled = fe.copy()
scaled[num_cols] = scaled_vals
scaled.head()

In [ ]:
# Plot 1: Boxplot of sepal length (after imputation)
plt.figure()
imputed['sepal length (cm)'].plot(kind='box')
plt.title('Boxplot: Sepal Length (cm)')
plt.ylabel('sepal length (cm)')
plt.show()

In [ ]:
# Plot 2: Scatter of petal length vs width colored by species via labels
plt.figure()
for sp, sub in imputed.groupby('species'):
    plt.scatter(sub['petal length (cm)'], sub['petal width (cm)'], label=sp, alpha=0.7)
plt.title('Petal Length vs Width by Species')
plt.xlabel('petal length (cm)')
plt.ylabel('petal width (cm)')
plt.legend()
plt.show()

In [ ]:
# Save a cleaned/engineered version for downstream modeling
CLEAN_PATH = "ml_projects/data/iris_cleaned_engineered.csv"
scaled.to_csv(CLEAN_PATH, index=False)
print("Saved:", CLEAN_PATH)

## Joins / Merges Example
We'll use two small CSVs: `employees.csv` and `departments.csv`.

In [ ]:
import pandas as pd
EMP_PATH = "ml_projects/data/employees.csv"
DEPT_PATH = "ml_projects/data/departments.csv"

emp = pd.read_csv(EMP_PATH)
dept = pd.read_csv(DEPT_PATH)
display(emp)
display(dept)

In [ ]:
# Inner join on dept_id
joined = emp.merge(dept, on='dept_id', how='inner')
joined

In [ ]:
# Simple groupby after join: avg salary by department
avg_sal = joined.groupby('dept_name')['salary'].mean().reset_index()
avg_sal